# Generate response using Model and Cosine Simillarity 
### 1. Predict class of user's input using model
### 2. Generate appropriate response using Cosine Simillarity within predicted class 

### Including necessary libraries 

In [1]:
# things we need for NLP
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

import pickle
import json



curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


### Restore Training data

In [2]:
# restore all of our data structures
data = pickle.load( open( "Processed_Data/Jay_training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
with open('Processed_Data/intents.json') as json_data:
    intents = json.load(json_data)

### Build and Restore model from saved checkpoints 

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# load our saved model
model.load('./JAY_model.tflearn')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\LENOVO\Documents\Jay_jupyter_notebooks\Simple-Chatbot-NLTK-Tensorflow\JAY_model.tflearn


### Clean and formatting user input 

In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

### Find cosine simillarity between user input and its category samples for fetching appropriate response 

In [9]:
def index_from_cosine_simillarity(sentence, patterns):
    sent_vec = bow(sentence, words)
    simil_list = []
    for i in range(len(patterns)):
        pat_vec = bow(patterns[i], words)
        simil_list.append(cosine_similarity([sent_vec],[pat_vec]))
    a = simil_list.index(max(simil_list))
    print ("Maximum simillarity score: \n=> ",max(simil_list))
    print ("--------------------------------------------------")
    print ("Nearer Question from Pre-train dataset of predicted category::\n=> ", patterns[a])
    print ("--------------------------------------------------")
    return a

### Classify user input and generate response 

In [11]:
# create a data structure to hold user context
context = {}
sample_reponses = None

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        index_value = index_from_cosine_simillarity(sentence, i['patterns'])
                        return print ('BOT Response:\n=> ',i['responses'][index_value])
                        # return print(random.choice(i['responses']))

            results.pop(0)

### sample user input with its response 

In [12]:
response('tell me film which has song Springtime')

Maximum simillarity score: 
=>  [[0.75]]
--------------------------------------------------
Nearer Question from Pre-train dataset of predicted category::
=>  which film had song springtime for hitler
--------------------------------------------------
BOT Response:
=>  The Producers


In [13]:
response('100 Groschen comming under which currency')

Maximum simillarity score: 
=>  [[0.5]]
--------------------------------------------------
Nearer Question from Pre-train dataset of predicted category::
=>  what currency consists of 100 groschen
--------------------------------------------------
BOT Response:
=>  Austrian Schilling


In [14]:
response('city which is heated by volcanic springs')

Maximum simillarity score: 
=>  [[0.79056942]]
--------------------------------------------------
Nearer Question from Pre-train dataset of predicted category::
=>  what world capital city is heated by volcanic springs
--------------------------------------------------
BOT Response:
=>  Reykjavik (Iceland)
